### 함수정리

In [63]:
def update_crawl(symbol, crawl_start_date) :
    
    try : 
        starting = time.time()

        ep='https://api.binance.com'
        candle = '/api/v3/klines'

        start_date = []
        start_time = []
        openn = []
        high = []
        low = []
        close = []
        volume = []
        #future_close.index[-1].timestamp() * 1000
        #int(str(future_close.index[-1].timestamp() * 1000)[:-2])
        #math.floor(time.mktime(future_close.index[-1].timetuple()) * 1000)
        start = math.floor(time.mktime(crawl_start_date.timetuple()) * 1000)
        first_params_candle = {'symbol': symbol, 'interval' : '1m','startTime' : start , 'limit' : 1 } # 초기세팅
        r1 = requests.get(ep+candle, params=first_params_candle )

        while len(r1.json()) > 0 :
            first_params_candle = {'symbol': symbol, 'interval' : '1m','startTime' : start , 'limit' : 1000 }
            r1 = requests.get(ep+candle, params=first_params_candle ) #use parameter
            req = r1.json()
            for i in range(0,len(req)) : 
                start_date.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%Y%m%d'))
                start_time.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%H%M'))
                close.append(req[i][4])
            if len(req) > 0 : # 받아온 데이터가 있을 경우 => 다음 루프 시작
                start = req[-1][6]+1
            else : print(symbol + " 다운로드 완료")
            #time.sleep(0.25) # 횟수제한을 위해 추가

        ticker = [symbol] * len(start_date)

        chartData = {'date': start_date, 'time': start_time, 'ticker': ticker, 'close': close}

        df = pd.DataFrame(chartData, columns=['date', 'time', 'ticker', 'close'])
        df2 = df[['date', 'time', 'close']]
        df2.columns = ['date', 'time', symbol]

        print("소요 시간 : ", time.time() - starting)
        
        return df2
    
    # 데이터 안불러 와지는 경우 keyerror가 나므로 예외처리함
    except KeyError as e :
        print(e)

In [64]:
def get_mom_h_l_tlist(frequency, look_period) :

    future_ret  = future_close.resample('{}T'.format(frequency)).last().pct_change().dropna()
    lookback_ret = future_ret.iloc[-look_period:].add(1).cumprod().iloc[-1]
    
    low_mom = lookback_ret[lookback_ret < float(lookback_ret.quantile([0.1]))].index.tolist()
    high_mom = lookback_ret[lookback_ret > float(lookback_ret.quantile([0.9]))].index.tolist()

    return low_mom, high_mom

In [65]:
def trade_print(trade_num, single_price, leverage, asset_num) :
    
    print('### trade number {} ###'.format(trade_num))
    print('')
    print('총 소비금액 : {}'.format(single_price / leverage * asset_num))
    print('')
    print('low_mom_asset')
    for i in range(3) :
        print(str(future_close[low_mom].iloc[-1].index[i]) + ' ' + str(single_price) + ' size 매수')

    print('')
    print('high_mom_asset')
    for i in range(3) :
        print(str(future_close[high_mom].iloc[-1].index[i]) + ' ' + str(single_price) + ' size 매도')

In [66]:
def time_print(trade_num) :
    print('### trade number {} ###'.format(trade_num))
    print('(start)          ~ (end)')
    hold_start = ( future_close.index[-1] + timedelta(minutes = 1) )
    hold_end = ( hold_start + timedelta(minutes = 60) )

    hold_start = hold_start.strftime('%Y-%m-%d %H:%M')
    hold_end = hold_end.strftime('%Y-%m-%d %H:%M')

    print(hold_start + ' ~ ' + hold_end)

### 모듈 import

In [67]:
import requests
from datetime import datetime
import pandas as pd
import time
import ccxt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from datetime import timedelta
import math

### 데이터 전처리 & 데이터 업데이트

In [68]:
raw_data = pd.read_csv('future_close_20200901_1226.csv')

In [69]:
raw_data['Date'] = raw_data['Date'].map(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
raw_data.set_index('Date', inplace = True)

future_close = raw_data.copy()
future_list = future_close.columns.tolist()
small_future_list = future_close.iloc[-1][future_close.iloc[-1] < 1].index.tolist()
future_close = future_close[small_future_list]

update_crawl_date = future_close.index[-1] + timedelta(minutes = 1)
update_crawl_date

update_all = update_crawl(small_future_list[0], update_crawl_date)
update_all = update_all.set_index('date')

small_future_list2 = small_future_list.copy()
small_future_list2.remove(small_future_list[0])

CVCUSDT 다운로드 완료
소요 시간 :  0.7410175800323486


In [70]:
for symbol in small_future_list2 :
    try : 
        tmp = update_crawl(symbol, update_crawl_date)
        update_all[symbol] = tmp.iloc[:len(update_all), 2].tolist()

    # 데이터 없을 경우 tmp가 none type이 되므로 attributeerror가 나게 됨
    except AttributeError as e:
        print(e)

    except ConnectionError as e:
        print(e)

    except ValueError as e:
        print(e)

    except TimeoutError as e:
        print(e)

KNCUSDT 다운로드 완료
소요 시간 :  0.7569770812988281
ENJUSDT 다운로드 완료
소요 시간 :  0.763958215713501
ZRXUSDT 다운로드 완료
소요 시간 :  0.7940421104431152
IOTAUSDT 다운로드 완료
소요 시간 :  0.8081786632537842
ADAUSDT 다운로드 완료
소요 시간 :  0.8197016716003418
IOSTUSDT 다운로드 완료
소요 시간 :  0.771937370300293
BZRXUSDT 다운로드 완료
소요 시간 :  0.8706793785095215
BATUSDT 다운로드 완료
소요 시간 :  0.8010070323944092
ALGOUSDT 다운로드 완료
소요 시간 :  0.7556991577148438
DOGEUSDT 다운로드 완료
소요 시간 :  0.8497302532196045
RLCUSDT 다운로드 완료
소요 시간 :  0.8248326778411865
TRXUSDT 다운로드 완료
소요 시간 :  0.7759368419647217
STORJUSDT 다운로드 완료
소요 시간 :  0.829780101776123
LRCUSDT 다운로드 완료
소요 시간 :  0.6841714382171631
XLMUSDT 다운로드 완료
소요 시간 :  0.7789163589477539
MATICUSDT 다운로드 완료
소요 시간 :  0.828782320022583
RSRUSDT 다운로드 완료
소요 시간 :  0.7978692054748535
ICXUSDT 다운로드 완료
소요 시간 :  0.6971356868743896
BLZUSDT 다운로드 완료
소요 시간 :  0.6754875183105469
RENUSDT 다운로드 완료
소요 시간 :  0.8247945308685303
FTMUSDT 다운로드 완료
소요 시간 :  0.7230789661407471
TOMOUSDT 다운로드 완료
소요 시간 :  0.741018533706665
XRPUSDT 다운로드 완료
소요 시간 :  0.

In [71]:
update_all = update_all.reset_index()
update_all['Date'] = update_all['date'] + update_all['time']
update_all['Date'] = update_all['Date'].map(lambda x : datetime.strptime(x, '%Y%m%d%H%M'))
update_all.set_index('Date', inplace = True)
update_all = update_all.iloc[:,2:]
update_all = update_all.apply(pd.to_numeric)

future_close = pd.concat([future_close,update_all])

### get high, low momentum aseet 

In [72]:
(low_mom, high_mom) = get_mom_h_l_tlist(10,6)

In [73]:
# (trade_num, single_price, leverage, asset_num)
trade_print(1, 3, 5, 6)
print('')
time_print(1)

### trade number 1 ###

총 소비금액 : 3.5999999999999996

low_mom_asset
STORJUSDT 3 size 매수
BLZUSDT 3 size 매수
OCEANUSDT 3 size 매수

high_mom_asset
MATICUSDT 3 size 매도
RENUSDT 3 size 매도
XRPUSDT 3 size 매도

### trade number 1 ###
(start)          ~ (end)
2020-12-28 20:19 ~ 2020-12-28 21:19
